# Introduction

Trying to submit some test data to the DCC.

This attempt is using some example files given to me by Henry.

I have a cell by gene quantification matrix and a cell location matrix.

In my first attempted I also posted a processed image henry generated which was constructed from several different channels of the tiff.


In [1]:
from pathlib import Path
import sys

#!{sys.executable} -m pip install encoded_client
EC = Path("~/proj/encoded_client").expanduser()
if EC not in sys.path:
    sys.path.append(str(EC))

In [2]:
from contextlib import contextmanager
import requests
from urllib import parse
import logging
from webdav3.client import Client
from encoded_client.encoded import (
    DCCValidator, 
    Document, 
    ENCODED, 
    HTTPError,
    LOGGER as encoded_logger,
)
from encoded_client.submission import run_aws_cp
import hashlib
import pandas
from pathlib import Path
from pprint import pprint, pformat
from tqdm import tqdm
import warnings
import shutil

In [3]:
ls *.tif

ls: cannot access '*.tif': No such file or directory


In [4]:
#server = ENCODED("encm-89-s-4ca7459c6-khine.demo.encodedcc.org")
#server = ENCODED("v129rc1.demo.encodedcc.org")
#server = ENCODED("test.encodedcc.org")
server = ENCODED("www.encodeproject.org")
server.load_netrc()
assert server.auth is not None
validator = DCCValidator(server)

In [5]:
def make_donor_alias(slide):
    rush_id = Path(slide).stem.split("_")[0]
    donor_alias = "john-stamatoyannopoulos:E{}".format(rush_id)
    return donor_alias

In [6]:
def get_encode_id(alias):
    level = encoded_logger.level
    encoded_logger.setLevel(logging.ERROR)
    try:
        document = server.get_json(alias)
        return document["@id"]
    except HTTPError as e:
        if e.response.status_code == 404:
            return None
        else:
            print(log)
            raise e
    finally:
        encoded_logger.setLevel(level)


In [7]:
# generate token from nextcloud security add app. (then delete it when done.)

options = {
 'webdav_hostname': "https://woldlab.caltech.edu/nextcloud/remote.php/dav/files/97213b24-42dd-1031-975c-63cbbc0afea6/",
 'webdav_login':    "diane",
 'webdav_password': "pWGxm-qSCMi-Hf2bB-GYdqW-SNgxW",
}
assert options["webdav_password"] is not None, "Assign temporary password from nextcloud"

nextcloud = Client(options)

In [8]:
listing = nextcloud.list("/rush_histology_slides/", get_info=True)

root_dir = Path("/rush_histology_slides/")
listing = listing[1:]

In [9]:
for row in tqdm(listing):
    dav_name = Path(row["path"][len("/nextcloud/remote.php/dav/files/97213b24-42dd-1031-975c-63cbbc0afea6"):])
    slide_name = Path(dav_name.name)
    # = root_dir / slide_name
    donor_alias = make_donor_alias(slide_name)
    donor_id = get_encode_id(donor_alias)
    if donor_id is None:
        print("{} does not exist.".format(donor_alias))
        continue

100%|██████████| 117/117 [00:32<00:00,  3.59it/s]


In [10]:
resubmit_all = Path("~/proj/flowcells/stranded-24038-24534-rush-resubmit-all.xlsx").expanduser()

In [11]:
rush_resubmit_book = pandas.ExcelFile(resubmit_all)
rush_resubmit_biosample = rush_resubmit_book.parse("Biosample")
rush_resubmit_biosample.head()

,uuid,accession,rush_id:skip,age:skip,age_units:skip,sex:skip,education:skip,cogdx:skip,clinical_status:skip,disease_term_name:skip,...,biosample_term_name:skip,aliases:array,organism,source,donor,donor_accession:skip,lab,award,nih_institutional_certification,disease_term_id:skip
0,801ba381-6ee9-4633-bf42-9ea31f835ad0,ENCBS075IEY,E3453103,90 or above,year,female,23,1,NCI,NaN,...,mid frontal cortex,barbara-wold:ENC4_cDNA649,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E3453103,ENCDO006DAA,barbara-wold,UM1HG009443,NaN,NaN
1,a6076606-2b3b-4b5e-99bb-8c6ab5125147,ENCBS742MVD,E4163791,90 or above,year,female,16,1,NCI,NaN,...,mid frontal cortex,barbara-wold:ENC4_cDNA650,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E4163791,ENCDO250PFZ,barbara-wold,UM1HG009443,NaN,NaN
2,ab5a3aeb-ab2c-479f-a4d6-28ba6e24a178,ENCBS819UHF,E4163791,90 or above,year,female,16,1,NCI,NaN,...,mid frontal cortex,barbara-wold:ENC4_cDNA665,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E4163791,ENCDO250PFZ,barbara-wold,UM1HG009443,NaN,NaN
3,ed956add-e8b9-47dd-8bdc-9b944dde5847,ENCBS644WSW,E4368365,90 or above,year,male,13,4,NINCDS PROB AD,"[""Alzheimer's disease""]",...,mid frontal cortex,barbara-wold:ENC4_cDNA651,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E4368365,ENCDO197QVG,barbara-wold,UM1HG009443,NaN,['DOID:10652']
4,80fe3505-9c40-4531-98f7-be875b3c11a6,ENCBS436WHA,E4933693,90 or above,year,female,16,4,NINCDS PROB AD,"[""Alzheimer's disease""]",...,mid frontal cortex,barbara-wold:ENC4_cDNA652,/organisms/human/,/sources/rush-university/,john-stamatoyannopoulos:E4933693,ENCDO258GJF,barbara-wold,UM1HG009443,NaN,['DOID:10652']


In [12]:
@contextmanager
def cache_nextcloud_file(nextcloud, pathname):
    pathname = Path(pathname)
    try:
        local_file = Path(pathname.name)
        nextcloud.download_file(str(pathname), local_file)
        yield local_file
    finally:
        if local_file.exists():
            local_file.unlink()

In [13]:
def make_slide_alias(slide):
    slide = Path(slide)
    return "barbara-wold:histology_{}".format(slide.stem)

In [14]:
def create_document(slide, slide_id, slide_alias):
    if slide_id is None:
        slide_doc = Document(
            slide,
            document_type="high resolution pathology slide image",
            aliases=[slide_alias],
            description="fresh frozen section, 30 microns, post fixed in ethanol. stain: blue = nuclei, pink = white matter",
            server=server,
        )

        doc = slide_doc.create_if_needed(server, slide_id, validator)
        with open("submitting-encode-rush-histology.log", "at") as log:
            log.write(pformat(doc))
        if "@graph" in doc:
            doc = doc["@graph"][0]
        slide_id = doc["@id"]

    return slide_id

In [15]:
def link_document(sheet, biosample_accession, document_id):
    """Look up document id in biosample documents to see if we need to add it"""
    biosample = server.get_json(biosample_accession, frame="object")
    if document_id not in biosample["documents"]:
        documents = biosample["documents"]
        documents.append(slide_id)
        return (biosample["@id"], documents)

In [16]:
bigones = [
    "barbara-wold:histology_2504543_5X_DLPFC.",
    "barbara-wold:histology_4368365_5X_DLPFC",
    "barbara-wold:histology_4933693_5X_DLPFC",
    "barbara-wold:histology_5115600_5X_DLPFC",
    "barbara-wold:histology_6735279_5X_DLPFC",
    "barbara-wold:histology_7404786_5X_DLPFC",
    "barbara-wold:histology_8282759_5X_DLPFC",
    "barbara-wold:histology_9972216_5X_DLPFC",
]

to_update = []
for row in tqdm(listing):
    dav_name = Path(row["path"][len("/nextcloud/remote.php/dav/files/97213b24-42dd-1031-975c-63cbbc0afea6"):])
    slide_name = Path(dav_name.name)
    # = root_dir / slide_name
    donor_alias = make_donor_alias(slide_name)
    donor_id = get_encode_id(donor_alias)
    if donor_id is None:
        print("{} does not exist.".format(donor_alias))
        continue

    slide_alias = make_slide_alias(slide_name)

    slide_id = get_encode_id(slide_alias)

    # hack to just test submitting the largest files
    #if slide_alias not in bigones:
    #    continue    
    
    try:
        if slide_id is None:
            with cache_nextcloud_file(nextcloud, dav_name) as local_name:
                local_size = local_name.stat().st_size
                remote_size = int(row["size"])
                if local_size != remote_size:
                    print("Different sizes {} {} != {}".format(local_name, local_size, remote_size))
                slide_id = create_document(local_name, slide_id, slide_alias)
        assert slide_id is not None

        for i, slide_row in rush_resubmit_biosample[rush_resubmit_biosample["donor"] == donor_alias].iterrows():
            links = link_document(rush_resubmit_biosample, slide_row.accession, slide_id)
            if links is not None:
                to_update.append(links)
    except HTTPError as e:
        print("problem with {} status {}".format(slide_name, e.response.status_code))
        


100%|██████████| 117/117 [44:27<00:00, 22.80s/it]


In [17]:
to_update

[('/biosamples/ENCBS885BFB/',
  ['/documents/49c22437-267b-463d-b6d5-6662221e67c9/']),
 ('/biosamples/ENCBS372KDQ/',
  ['/documents/a2bf5f19-60ae-44e0-889d-ce3b73a62759/']),
 ('/biosamples/ENCBS875BBD/',
  ['/documents/f11423bf-03f6-4197-b4e3-8cc71e6096d1/']),
 ('/biosamples/ENCBS137SYN/',
  ['/documents/ed86fef7-3ae6-4760-9f28-b3275ffa2ee7/']),
 ('/biosamples/ENCBS934UCP/',
  ['/documents/13bc89c5-9a78-46d2-a2ab-a9f24cf74a64/']),
 ('/biosamples/ENCBS765SSW/',
  ['/documents/4635d4a5-56c0-4ecc-981c-38de36cd0f90/']),
 ('/biosamples/ENCBS735JRG/',
  ['/documents/497cf1c7-54c4-4ff9-91b3-142e26c8b44c/']),
 ('/biosamples/ENCBS352YCX/',
  ['/documents/79617662-2862-4614-b2b4-9de895c19b82/']),
 ('/biosamples/ENCBS168FVU/',
  ['/documents/87e4ab2e-219d-4127-b3bd-729d4ca875bc/']),
 ('/biosamples/ENCBS870MVU/',
  ['/documents/cde5a0d4-21b2-4c1a-9a1f-2b936f8a038a/']),
 ('/biosamples/ENCBS492ELW/',
  ['/documents/5f84ed6e-1292-493b-a59b-2f9e4932784f/']),
 ('/biosamples/ENCBS634VGA/',
  ['/document

In [18]:
update_df = pandas.DataFrame(to_update, columns=["biosample", "documents"])
update_df

,biosample,documents
0,/biosamples/ENCBS885BFB/,[/documents/49c22437-267b-463d-b6d5-6662221e67...
1,/biosamples/ENCBS372KDQ/,[/documents/a2bf5f19-60ae-44e0-889d-ce3b73a627...
2,/biosamples/ENCBS875BBD/,[/documents/f11423bf-03f6-4197-b4e3-8cc71e6096...
3,/biosamples/ENCBS137SYN/,[/documents/ed86fef7-3ae6-4760-9f28-b3275ffa2e...
4,/biosamples/ENCBS934UCP/,[/documents/13bc89c5-9a78-46d2-a2ab-a9f24cf74a...
...,...,...
116,/biosamples/ENCBS069FLE/,[/documents/bacc33eb-8f72-41d7-908b-28d3a7169f...
117,/biosamples/ENCBS135ECM/,[/documents/89a814cd-a7c3-4d6d-a895-a0c70d3553...
118,/biosamples/ENCBS253LWF/,[/documents/e20fa099-c570-4a7f-8242-6c1dcd90bf...
119,/biosamples/ENCBS687KKD/,[/documents/e3d99a3d-30de-49d9-af9b-e2d7bbf5a5...


In [19]:
update_df.to_csv("rush-histology.csv", index=False)

In [20]:
print(update_df.to_csv(index=False))

biosample,documents
/biosamples/ENCBS885BFB/,['/documents/49c22437-267b-463d-b6d5-6662221e67c9/']
/biosamples/ENCBS372KDQ/,['/documents/a2bf5f19-60ae-44e0-889d-ce3b73a62759/']
/biosamples/ENCBS875BBD/,['/documents/f11423bf-03f6-4197-b4e3-8cc71e6096d1/']
/biosamples/ENCBS137SYN/,['/documents/ed86fef7-3ae6-4760-9f28-b3275ffa2ee7/']
/biosamples/ENCBS934UCP/,['/documents/13bc89c5-9a78-46d2-a2ab-a9f24cf74a64/']
/biosamples/ENCBS765SSW/,['/documents/4635d4a5-56c0-4ecc-981c-38de36cd0f90/']
/biosamples/ENCBS735JRG/,['/documents/497cf1c7-54c4-4ff9-91b3-142e26c8b44c/']
/biosamples/ENCBS352YCX/,['/documents/79617662-2862-4614-b2b4-9de895c19b82/']
/biosamples/ENCBS168FVU/,['/documents/87e4ab2e-219d-4127-b3bd-729d4ca875bc/']
/biosamples/ENCBS870MVU/,['/documents/cde5a0d4-21b2-4c1a-9a1f-2b936f8a038a/']
/biosamples/ENCBS492ELW/,['/documents/5f84ed6e-1292-493b-a59b-2f9e4932784f/']
/biosamples/ENCBS634VGA/,['/documents/59a02bbe-07f1-4b9f-8d4b-4bc41633c8e7/']
/biosamples/ENCBS542OFI/,['/documents/9e1f16